In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,1,32,1,41.0
6132,0,28,1,30.0
5986,0,44,1,38.0
7682,0,34,1,38.0
4910,1,31,1,40.0


In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 262us/sample - loss: 1.6354 - mae: 0.9961 - mse: 1.6354 - val_loss: 1.1975 - val_mae: 0.8424 - val_mse: 1.1975
Epoch 2/10
6670/6670 [==============================] - 1s 145us/sample - loss: 1.4757 - mae: 0.9556 - mse: 1.4757 - val_loss: 1.1218 - val_mae: 0.8134 - val_mse: 1.1218
Epoch 3/10
6670/6670 [==============================] - 1s 149us/sample - loss: 1.4264 - mae: 0.9350 - mse: 1.4264 - val_loss: 1.4492 - val_mae: 0.9400 - val_mse: 1.4492
Epoch 4/10
6670/6670 [==============================] - 1s 161us/sample - loss: 1.3757 - mae: 0.9189 - mse: 1.3757 - val_loss: 1.3091 - val_mae: 0.9132 - val_mse: 1.3091
Epoch 5/10
6670/6670 [==============================] - 1s 132us/sample - loss: 1.2890 - mae: 0.8907 - mse: 1.2890 - val_loss: 1.1467 - val_mae: 0.8223 - val_mse: 1.1467
Epoch 6/10
6670/6670 [==============================] - 1s 140us/sample - loss: 1.2876 - mae: 0.8906 - 

In [13]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [14]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.1610103
Actual val:  6.7792145564999995

Predicted val:  7.1844554
Actual val:  9.56365292556

Predicted val:  7.2774763
Actual val:  7.936641432

Predicted val:  6.7271967
Actual val:  5.8753192823

Predicted val:  6.9000525
Actual val:  6.3118345610599995

Predicted val:  7.5965657
Actual val:  6.6248909731

Predicted val:  6.4059653
Actual val:  7.06361087448

Predicted val:  6.9237285
Actual val:  7.5618555866

Predicted val:  6.7500615
Actual val:  6.2501051276999995

Predicted val:  6.585125
Actual val:  7.62578964258

